# Mispriced Bonds

#### This program finds mispriced bonds using input spot curve.
#### Discount bonds using spot curve and compare them with market prices

1. Part 1 use linear interpolated spot curve using Spot_Curve_Linear_Interpolation.ipynb. Inital data from bonds.
2. Part 2 use linear interpolated spot curve. Initial data from swaps.
3. Part 3 use Cubic interpolated spot curve. Initial data from bonds.

In [8]:
import pandas as pd
import numpy as np

#### 1. Linear Interpolated Spot Curve

##### 1.1 Import spot curve data

In [9]:
adrs = 'SpotCurveLinear.csv'
spotcurve=pd.read_csv(adrs)
spotcurve

,Unnamed: 0,Maturity,Coupon,Ask Price,Year,Month,n,spot
0,1,2/15/2020,1.375000,100.0,2020,2,1,1.375000
1,2,8/15/2020,1.500000,99.984375,2020,8,2,1.500000
2,3,2/28/2021,1.125000,99.61328125,2021,2,3,1.125000
3,4,8/31/2021,1.500000,100.1328125,2021,8,4,1.500000
4,5,2/28/2022,1.750000,100.7890625,2022,2,5,1.750000
5,6,8/15/2022,1.500000,100.390625,2022,8,6,1.500000
6,7,2/28/2023,1.500000,100.5078125,2023,2,7,1.500000
7,8,8/31/2023,1.375000,100.140625,2023,8,8,1.375000
8,9,/,1.312500,/,2024,2,9,1.308265
9,10,8/31/2024,1.250000,99.5625,2024,8,10,1.250000


##### 1.2 Import bonds data

In [25]:
bond_adrs = '/Users/wanqiwang/Desktop/bond_data.csv'

bond = pd.read_csv(bond_adrs)

bond['Maturity'] = pd.to_datetime(bond['Maturity'], format='%m/%d/%Y')
bond = bond.sort_values('Maturity')
bond = bond.reset_index().iloc[:,1:]
bond['time'] = float(0)
for i in range(len(bond)): 
    bond['time'][i] = float((bond.Maturity[i]-bond.Maturity[0]).days)/365
bond = bond.loc[bond['time']>0.5]
bond = bond.loc[bond['time']<9.5]
bond = bond.reset_index().iloc[:,1:]
bond = bond[['Coupon','Maturity','Identifier','Ask Price','time']]
bond

/Users/wanqiwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,Coupon,Maturity,Identifier,Ask Price,time
0,2.625,2020-07-31,912828Y46,100.546875,0.542466
1,1.625,2020-07-31,912828XM7,100.050781,0.542466
2,2.000,2020-07-31,912828VP2,100.238281,0.542466
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562
4,8.750,2020-08-15,912810EG9,103.917969,0.583562
5,2.625,2020-08-15,912828NT3,100.582031,0.583562
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397
7,2.125,2020-08-31,912828VV9,100.332031,0.627397
8,1.375,2020-08-31,912828L32,99.906250,0.627397
9,1.375,2020-09-15,9128282V1,99.910156,0.668493


##### 1.3 Discount bonds using spot curve

Define a function return spot rate

In [26]:
def spot_func(x):
    if 0<x<1:
        return 2.1;
    return spotcurve.spot[int(x)]+(x%1)*(spotcurve.spot[int(x)+1]-spotcurve.spot[int(x)])

Discount, estimate price

In [27]:
bond['price_est']=0.0
for i in range(len(bond)):  # t=bond.loc[1]  i=2
    coupon_t=[(bond.time[i]*2)%1 + i for i in range(int(bond.time[i]*2)+1)]    
    bond['price_est'][i]=sum([bond.Coupon[i]/2*(1/(1+spot_func(j)/200))**j for j in coupon_t]+[100*(1/(1+spot_func(bond.time[i]*2)/200))**(bond.time[i]*2)])
bond

/Users/wanqiwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Coupon,Maturity,Identifier,Ask Price,time,price_est
0,2.625,2020-07-31,912828Y46,100.546875,0.542466,101.823090
1,1.625,2020-07-31,912828XM7,100.050781,0.542466,100.827485
2,2.000,2020-07-31,912828VP2,100.238281,0.542466,101.200837
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562,100.661120
4,8.750,2020-08-15,912810EG9,103.917969,0.583562,107.879254
5,2.625,2020-08-15,912828NT3,100.582031,0.583562,101.781175
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397,101.739236
7,2.125,2020-08-31,912828VV9,100.332031,0.627397,101.241434
8,1.375,2020-08-31,912828L32,99.906250,0.627397,100.494730
9,1.375,2020-09-15,9128282V1,99.910156,0.668493,100.458008


##### 1.4 Misprice

In [28]:
bond['Market_minus_estimate']=bond['Ask Price']-bond['price_est']
bond['misprice_percent']=bond.Market_minus_estimate/bond['Ask Price']*100
bond['misprice']=bond['misprice_percent']/abs(bond['misprice_percent'])
print('standard deviation of misprice percentile from market price:',np.std(bond['misprice_percent']))

standard deviation of misprice percentile from market price: 0.6540366962142119


In [29]:
bond

,Coupon,Maturity,Identifier,Ask Price,time,price_est,Market_minus_estimate,misprice_percent,misprice
0,2.625,2020-07-31,912828Y46,100.546875,0.542466,101.823090,-1.276215,-1.269274,-1.0
1,1.625,2020-07-31,912828XM7,100.050781,0.542466,100.827485,-0.776704,-0.776310,-1.0
2,2.000,2020-07-31,912828VP2,100.238281,0.542466,101.200837,-0.962556,-0.960268,-1.0
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562,100.661120,-0.676745,-0.676850,-1.0
4,8.750,2020-08-15,912810EG9,103.917969,0.583562,107.879254,-3.961285,-3.811935,-1.0
5,2.625,2020-08-15,912828NT3,100.582031,0.583562,101.781175,-1.199144,-1.192205,-1.0
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397,101.739236,-1.106424,-1.099466,-1.0
7,2.125,2020-08-31,912828VV9,100.332031,0.627397,101.241434,-0.909403,-0.906393,-1.0
8,1.375,2020-08-31,912828L32,99.906250,0.627397,100.494730,-0.588480,-0.589033,-1.0
9,1.375,2020-09-15,9128282V1,99.910156,0.668493,100.458008,-0.547851,-0.548344,-1.0


#### 2. Linear Interpolated Spot Curve Using Swaps

##### 2.1 Import spot curve data

In [45]:
adrs_spot_s = 'SpotCurveSwaps.csv'
swap=pd.read_csv(adrs_spot_s)
swap.index+=1  # take care of this
swap = swap[['n','spot']]
swap.dropna(inplace=True)
swap

,n,spot
1,0.0,1.375000
2,1.0,1.490554
3,2.0,1.603863
4,3.0,1.519887
5,4.0,1.435308
6,5.0,1.405622
7,6.0,1.375740
8,7.0,1.368360
9,8.0,1.360892
10,9.0,1.364759


##### 2.2 Import bonds data

In [48]:

bond_swap=pd.read_csv(bond_adrs)

bond_swap['Maturity']=pd.to_datetime(bond_swap['Maturity'], format='%m/%d/%Y')
bond_swap=bond_swap.sort_values('Maturity')
bond_swap=bond_swap.reset_index().iloc[:,1:]
bond_swap['time']=float(0)
for i in range(len(bond_swap)):  #i=311
    bond_swap['time'][i]=float((bond_swap.Maturity[i]-bond_swap.Maturity[0]).days)/365
bond_swap=bond_swap.loc[bond_swap['time']>0.5]
bond_swap=bond_swap.loc[bond_swap['time']<29.5]
bond_swap=bond_swap.reset_index().iloc[:,1:]
bond_swap = bond_swap[['Coupon','Maturity','Identifier','Ask Price','time']]
bond_swap

/Users/wanqiwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Coupon,Maturity,Identifier,Ask Price,time
0,2.625,2020-07-31,912828Y46,100.546875,0.542466
1,1.625,2020-07-31,912828XM7,100.050781,0.542466
2,2.000,2020-07-31,912828VP2,100.238281,0.542466
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562
4,8.750,2020-08-15,912810EG9,103.917969,0.583562
5,2.625,2020-08-15,912828NT3,100.582031,0.583562
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397
7,2.125,2020-08-31,912828VV9,100.332031,0.627397
8,1.375,2020-08-31,912828L32,99.906250,0.627397
9,1.375,2020-09-15,9128282V1,99.910156,0.668493


##### 2.3 Discount bonds using spot curve

Define a function return spot rate

In [49]:
def spot_func_swap(x):
    if 0<x<1:
        return 2.7260;
    return swap.spot[int(x)]+(x%1)*(swap.spot[int(x)+1]-swap.spot[int(x)])

Discount, estimate price

In [50]:
bond_swap['price_est']=0.0
for i in range(len(bond_swap)):  # t=bond_swap.loc[1]  i=2
    coupon_t=[(bond_swap.time[i]*2)%1 + i for i in range(int(bond_swap.time[i]*2)+1)]    
    bond_swap['price_est'][i]=sum([bond_swap.Coupon[i]/2*(1/(1+spot_func_swap(j)/200))**j for j in coupon_t]+[100*(1/(1+spot_func_swap(bond_swap.time[i]*2)/200))**(bond_swap.time[i]*2)])
bond_swap

/Users/wanqiwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Coupon,Maturity,Identifier,Ask Price,time,price_est
0,2.625,2020-07-31,912828Y46,100.546875,0.542466,101.867872
1,1.625,2020-07-31,912828XM7,100.050781,0.542466,100.872176
2,2.000,2020-07-31,912828VP2,100.238281,0.542466,101.245562
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562,100.685979
4,8.750,2020-08-15,912810EG9,103.917969,0.583562,107.904777
5,2.625,2020-08-15,912828NT3,100.582031,0.583562,101.806137
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397,101.739482
7,2.125,2020-08-31,912828VV9,100.332031,0.627397,101.241634
8,1.375,2020-08-31,912828L32,99.906250,0.627397,100.494862
9,1.375,2020-09-15,9128282V1,99.910156,0.668493,100.431620


##### 2.4 Misprice

In [51]:
bond_swap['Market_minus_estimate']=bond_swap['Ask Price']-bond_swap['price_est']
bond_swap['misprice_percent']=bond_swap.Market_minus_estimate/bond_swap['Ask Price']*100
bond_swap['misprice']=bond_swap['misprice_percent']/abs(bond_swap['misprice_percent'])
print('standard deviation of misprice percentile from market price:',np.std(bond_swap['misprice_percent']))

standard deviation of misprice percentile from market price: 0.6184413042216219


In [52]:
bond_swap

,Coupon,Maturity,Identifier,Ask Price,time,price_est,Market_minus_estimate,misprice_percent,misprice
0,2.625,2020-07-31,912828Y46,100.546875,0.542466,101.867872,-1.320997,-1.313813,-1.0
1,1.625,2020-07-31,912828XM7,100.050781,0.542466,100.872176,-0.821395,-0.820978,-1.0
2,2.000,2020-07-31,912828VP2,100.238281,0.542466,101.245562,-1.007281,-1.004887,-1.0
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562,100.685979,-0.701604,-0.701714,-1.0
4,8.750,2020-08-15,912810EG9,103.917969,0.583562,107.904777,-3.986808,-3.836496,-1.0
5,2.625,2020-08-15,912828NT3,100.582031,0.583562,101.806137,-1.224106,-1.217023,-1.0
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397,101.739482,-1.106670,-1.099711,-1.0
7,2.125,2020-08-31,912828VV9,100.332031,0.627397,101.241634,-0.909603,-0.906593,-1.0
8,1.375,2020-08-31,912828L32,99.906250,0.627397,100.494862,-0.588612,-0.589164,-1.0
9,1.375,2020-09-15,9128282V1,99.910156,0.668493,100.431620,-0.521464,-0.521933,-1.0


#### 3. Cubic Interpolated Spot Curve

##### 3.1 Import spot curve data

In [32]:
bond_adrs_c = 'spot_cubic.csv'
cubic=pd.read_csv(bond_adrs_c)

cubic.index+=1  # take care of this
cubic = cubic[["YTM","spot"]]
cubic

,YTM,spot
1,0.5,1.500000
2,1.0,1.125000
3,1.5,1.500000
4,2.0,1.750000
5,2.5,1.500000
6,3.0,1.500000
7,3.5,1.375000
8,4.0,1.500000
9,4.5,1.250000
10,5.0,1.500000


##### 3.2 Import bonds data

In [35]:
bond_cubic = pd.read_csv(bond_adrs)

bond_cubic['Maturity']=pd.to_datetime(bond_cubic['Maturity'], format='%m/%d/%Y')
bond_cubic = bond_cubic.sort_values('Maturity')
bond_cubic = bond_cubic.reset_index().iloc[:,1:]
bond_cubic['time'] = float(0)
for i in range(len(bond_cubic)):
    bond_cubic['time'][i] = float((bond_cubic.Maturity[i]-bond_cubic.Maturity[0]).days)/365
bond_cubic = bond_cubic.loc[bond_cubic['time']>0.5]
bond_cubic = bond_cubic.loc[bond_cubic['time']<9.5]
bond_cubic = bond_cubic.reset_index().iloc[:,1:]
bond_cubic = bond_cubic[['Coupon','Maturity','Identifier','Ask Price','time']]
bond_cubic

/Users/wanqiwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Coupon,Maturity,Identifier,Ask Price,time
0,2.625,2020-07-31,912828Y46,100.546875,0.542466
1,1.625,2020-07-31,912828XM7,100.050781,0.542466
2,2.000,2020-07-31,912828VP2,100.238281,0.542466
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562
4,8.750,2020-08-15,912810EG9,103.917969,0.583562
5,2.625,2020-08-15,912828NT3,100.582031,0.583562
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397
7,2.125,2020-08-31,912828VV9,100.332031,0.627397
8,1.375,2020-08-31,912828L32,99.906250,0.627397
9,1.375,2020-09-15,9128282V1,99.910156,0.668493


##### 3.3 Discount bonds using spot curve

Define a function return spot rate

In [36]:
def spot_func_cubic(x):
    if 0<x<1:
        return 2.1;
    return cubic.spot[int(x)]+(x%1)*(cubic.spot[int(x)+1]-cubic.spot[int(x)])

Discount, estimate price

In [37]:
bond_cubic['price_est']=0.0
for i in range(len(bond_cubic)):  # t=bond_cubic.loc[1]  i=2
    coupon_t=[(bond_cubic.time[i]*2)%1 + i for i in range(int(bond_cubic.time[i]*2)+1)]    
    bond_cubic['price_est'][i]=sum([bond_cubic.Coupon[i]/2*(1/(1+spot_func_cubic(j)/200))**j for j in coupon_t]+[100*(1/(1+spot_func_cubic(bond_cubic.time[i]*2)/200))**(bond_cubic.time[i]*2)])
bond_cubic

/Users/wanqiwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Coupon,Maturity,Identifier,Ask Price,time,price_est
0,2.625,2020-07-31,912828Y46,100.546875,0.542466,101.823090
1,1.625,2020-07-31,912828XM7,100.050781,0.542466,100.827485
2,2.000,2020-07-31,912828VP2,100.238281,0.542466,101.200837
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562,100.661120
4,8.750,2020-08-15,912810EG9,103.917969,0.583562,107.879254
5,2.625,2020-08-15,912828NT3,100.582031,0.583562,101.781175
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397,101.739236
7,2.125,2020-08-31,912828VV9,100.332031,0.627397,101.241434
8,1.375,2020-08-31,912828L32,99.906250,0.627397,100.494730
9,1.375,2020-09-15,9128282V1,99.910156,0.668493,100.458008


##### 3.4 Misprice

In [38]:
bond_cubic['Market_minus_estimate']=bond_cubic['Ask Price']-bond_cubic['price_est']
bond_cubic['misprice_percent']=bond_cubic.Market_minus_estimate/bond_cubic['Ask Price']*100
bond_cubic['misprice']=bond_cubic['misprice_percent']/abs(bond_cubic['misprice_percent'])
print('standard deviation of misprice percentile from market price:',np.std(bond_cubic['misprice_percent']))

standard deviation of misprice percentile from market price: 0.6364824776641509


In [39]:
bond_cubic

,Coupon,Maturity,Identifier,Ask Price,time,price_est,Market_minus_estimate,misprice_percent,misprice
0,2.625,2020-07-31,912828Y46,100.546875,0.542466,101.823090,-1.276215,-1.269274,-1.0
1,1.625,2020-07-31,912828XM7,100.050781,0.542466,100.827485,-0.776704,-0.776310,-1.0
2,2.000,2020-07-31,912828VP2,100.238281,0.542466,101.200837,-0.962556,-0.960268,-1.0
3,1.500,2020-08-15,9128282Q2,99.984375,0.583562,100.661120,-0.676745,-0.676850,-1.0
4,8.750,2020-08-15,912810EG9,103.917969,0.583562,107.879254,-3.961285,-3.811935,-1.0
5,2.625,2020-08-15,912828NT3,100.582031,0.583562,101.781175,-1.199144,-1.192205,-1.0
6,2.625,2020-08-31,9128284Y3,100.632812,0.627397,101.739236,-1.106424,-1.099466,-1.0
7,2.125,2020-08-31,912828VV9,100.332031,0.627397,101.241434,-0.909403,-0.906393,-1.0
8,1.375,2020-08-31,912828L32,99.906250,0.627397,100.494730,-0.588480,-0.589033,-1.0
9,1.375,2020-09-15,9128282V1,99.910156,0.668493,100.458008,-0.547851,-0.548344,-1.0
